# Forms

Up to now, we have only implemented views that retrieve data from the database. Suppose that we want to allow users to update profiles (later we will talk about restricting this capability to certain users)

Django has the `forms.ModelForm` class for 

`clubmanager/forms.py`
```python
from django import forms
from clubmanager.models import Member

class MemberForm(forms.ModelForm):
    model = Member
```


## The view

`views.py`
```python
from clubmanager.forms import MemberForm

def member_update(request,member_id):
    form = MemberForm()
    # now we need to work out whether data is being submitted or not
```

## The template

`member_update.html`
```html
...
<form>

{% csrf_token %}

    {{ form }}
</form>
...
```
You can use `form.as_p` to have the form elements separated by `<p>` tags. 

The opening and closing `<form> ... </form>` tags
are not rendered by the ModelForm class, nor is the submit button.

**CSRF** = Cross Site Request Forgery

## The view (again)

A standard pattern is that the form is submitted back to the same URL from whence tit came. So 

* the form is initially loaded by a `GET` request to `http://example.com/form` 
* when the user clicks submit the data is submitted back to `http://example.com/form` by a `POST` request

We can implement this in the view as follows 

`views.py` 
```python
def member_update(request,member_id):
    if request.method == "POST":
        form = MemberForm(request.POST)
        if form.is_valid():
            # some code to update the Member object in the database
            return HttpResponse('Thanks')
        else:
            return HttpResponse('Invalid data')
    form = MemberForm()
    return render(request,'clubmanager/member_update.html',{
        'form':form
        })
```

This works, but there are several things that would improve it
* We would like a slick way to update the database with the form data
* We want a better flow when the form is submitted - the user should be directed back to the form if the data does not validate and directed back to the profile page if the data validates
* we want a link from the profile page to the update page.
* the form should be prepopulated with the existing data when we load it first